<a href="https://colab.research.google.com/github/anaayaoli/An-lisis-de-datos-conversacionales/blob/main/Notebooks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📌 Sección 1: Importar librerías y cargar datos
import pandas as pd

# Cargar archivo Excel con los datos de intenciones
df = pd.read_excel("../data/IA.xlsx")

# Vista previa de las primeras filas
df.head()

# ---
# 📌 Sección 2: Filtrar datos
# Nos quedamos solo con las filas donde el intent es el especificado
df_filtrado = df[df["Nombre de Intent"] == "0.0. Enviar mensaje a LLM Default"]

print("Número de filas filtradas:", df_filtrado.shape)
df_filtrado.head()

# Asumimos que la columna de texto se llama "Texto" (ajustar si cambia)
textos_entrada = df_filtrado["Texto"].dropna().tolist()

# ---
# 📌 Sección 3: Preprocesamiento con spaCy
!pip install -q spacy
!python -m spacy download es_core_news_sm -q

import spacy
import string

# Cargar modelo en español
nlp = spacy.load("es_core_news_sm")

# Función de limpieza y lematización
def limpiar_y_lemmatizar(texto):
    doc = nlp(texto.lower())  # pasar a minúsculas y procesar con spaCy
    tokens_limpios = [
        token.lemma_ for token in doc
        if token.is_alpha and not token.is_stop
    ]
    return tokens_limpios

# Aplicamos la función a todos los textos
textos_procesados = [limpiar_y_lemmatizar(texto) for texto in textos_entrada]

# ---
# 📌 Sección 4: Filtrar palabras irrelevantes
palabras_excluir = {
    "event", "detection", "eh", "querer", "salir", "llegar", "q", "ok", "favor",
    "bienvenida", "ayudar", "necesitar", "gracias", "pasar", "ayer",
    "esperar", "dejar", "hola", "dar", "bastante", "hacer"
}

textos_filtrados = [
    [token for token in texto if token not in palabras_excluir]
    for texto in textos_procesados
]

print("Texto original:", textos_entrada[0])
print("Texto procesado:", textos_filtrados[0])

# ---
# 📌 Sección 5: Frecuencia de palabras
from itertools import chain
from collections import Counter

# Aplanar todos los tokens
tokens_totales = list(chain.from_iterable(textos_filtrados))

# Contar palabras
frecuencia_palabras = Counter(tokens_totales)

# Mostrar top 20 palabras
frecuencia_palabras.most_common(20)

# ---
# 📌 Sección 6: Visualización de frecuencias
import matplotlib.pyplot as plt

palabras, frecuencias = zip(*frecuencia_palabras.most_common(25))
plt.figure(figsize=(10,5))
plt.bar(palabras, frecuencias)
plt.xticks(rotation=45)
plt.title("Top 25 palabras más frecuentes")
plt.show()

# ---
# 📌 Sección 7: Topic Modeling con LDA
from sklearn.feature_extraction.text import CountVectorizer

# Unimos los tokens por documento
documentos = [" ".join(texto) for texto in textos_filtrados]

vectorizer = CountVectorizer(max_df=0.95, min_df=2)
X = vectorizer.fit_transform(documentos)

from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(X)

# Función para mostrar palabras top por tema
def mostrar_top_palabras_por_tema(modelo, vectorizer, n_palabras=10):
    palabras = vectorizer.get_feature_names_out()
    for idx, tema in enumerate(modelo.components_):
        print(f"\n🔹 Tema {idx + 1}:")
        top_palabras = [palabras[i] for i in tema.argsort()[:-n_palabras - 1:-1]]
        print(", ".join(top_palabras))

mostrar_top_palabras_por_tema(lda, vectorizer)

# ---
# 📌 Sección 8: Visualización interactiva de temas
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, X, vectorizer)

In [ ]:
import pandas as pd
df = pd.read_excel("IA.xlsx")
df.head()
# Filtrar solo las filas con ese valor exacto en la columna "Nombre de Intent"
df_filtrado = df[df["Nombre de Intent"] == "0.0. Enviar mensaje a LLM Default"]
print(df_filtrado.shape)  # Ver cuántas filas quedaron
print(df_filtrado.head())  # Ver una muestra de las primeras filas
textos_entrada = df_filtrado["Texto de Entrada"].dropna().tolist()
print(textos_entrada[:10])
frases_irrelevantes = {
    "sí", "ok", "vale", "gracias", "muchas gracias", "por favor", "hola", "buenos días",
    "buenas", "buenas tardes", "buenas noches", "de acuerdo", "está bien", "dale", "quiero ayuda", "por favor"
}
def es_relevante(frase):
    frase_limpia = frase.strip().lower()
    # Filtrar frases demasiado cortas o vacías
    if len(frase_limpia.split()) <= 2:
        return False
    # Filtrar saludos, cortesías, frases vacías
    if frase_limpia in frases_irrelevantes:
        return False
    return True

import re

# 1. Eliminar entradas que contengan "event detection" (sin distinguir mayúsculas)
textos_filtrados = [
    texto for texto in textos_entrada
    if "event detection" not in texto.lower()
]

# 2. Función para limpieza mínima (minúsculas + sin signos)
def limpiar_frase(texto):
    texto = texto.lower()
    texto = re.sub(r'[^\w\sáéíóúüñ]', '', texto)  # elimina signos de puntuación pero mantiene letras con tildes y ñ
    return texto

# 3. Aplicar limpieza
frases_limpias = [limpiar_frase(texto) for texto in textos_filtrados]

!pip install bertopic

from bertopic import BERTopic

# Crear modelo (multilingual sirve para español)
modelo_topic = BERTopic(language="multilingual", verbose=True)

# Entrenar modelo
temas, probabilidades = modelo_topic.fit_transform(frases_limpias)

from bertopic import BERTopic
from hdbscan import HDBSCAN

# Crear un clusterizador que agrupe mejor (menos temas)
hdbscan_model = HDBSCAN(min_cluster_size=200, min_samples=1, metric='euclidean', prediction_data=True)

# Entrenar el modelo con menos fragmentación
modelo_topic = BERTopic(hdbscan_model=hdbscan_model, language="multilingual", verbose=True)
temas, probabilidades = modelo_topic.fit_transform(frases_limpias)

df_temas = modelo_topic.get_topic_info()
print(df_temas.head(10))  # Ver los 10 temas más frecuentes

modelo_topic.get_topic(-1)

# Por ejemplo, textos asignados al tema 1
indices_tema_1 = [i for i, t in enumerate(temas) if t == 1]

# Mostrar esos textos
for i in indices_tema_1[:5]:  # solo los primeros 5
    print(f"- {frases_limpias[i]}")
modelo_topic.visualize_topics()

modelo_topic.visualize_barchart(top_n_topics=42)

modelo_topic_reducido = modelo_topic.reduce_topics(
    frases_limpias,
    nr_topics=20  # for example, reduce to 6 main topics
)

# Get the new topic assignments for the documents using the reduced model
nuevo_temas, probabilidades = modelo_topic_reducido.transform(frases_limpias)

modelo_topic_reducido.visualize_barchart(top_n_topics=20)


In [ ]:
pip install bertopic hdbscan openpyxl pandas
import pandas as pd
from bertopic import BERTopic
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

# Cargar el archivo Excel
df = pd.read_excel("IA.xlsx")

# Filtrar por el intent del modelo LLM
df_filtrado = df[df["Nombre de Intent"] == "0.0. Enviar mensaje a LLM Default"]

# Eliminar entradas vacías y las que contengan "from_zendesk_full"
textos = df_filtrado["Texto de Entrada"].dropna()
textos = textos[~textos.str.contains("from_zendesk_full", case=False)]

# Convertir a lista
frases = textos.tolist()

# Filtrar frases muy cortas o irrelevantes
frases_limpias = [
    f for f in frases
    if isinstance(f, str)
    and len(f.split()) > 2
    and not any(p in f.lower() for p in ["gracias", "hola", "sí", "ok", "por favor", "event"])
]


# Crear clusterizador más estricto
hdbscan_model = HDBSCAN(min_cluster_size=200, min_samples=10, metric='euclidean', prediction_data=True)

# Crear modelo BERTopic
modelo_topic = BERTopic(hdbscan_model=hdbscan_model, language="multilingual", verbose=True)

# Entrenar modelo
temas, probs = modelo_topic.fit_transform(frases_limpias)

# Reducir número de temas (opcional: puedes ajustar `nr_topics`)
modelo_topic_reducido = modelo_topic.reduce_topics(frases_limpias, nr_topics=20)

# Obtener info de temas
df_temas = modelo_topic_reducido.get_topic_info()

# Eliminar temas irrelevantes (revisar manualmente con get_topic_info())
temas_a_eliminar = []

# Por ejemplo, elimina los que contengan palabras muy genéricas
palabras_irrelevantes = ["gracias", "hola", "ok", "por", "favor", "sí"]
for topic in df_temas["Topic"]:
    palabras_topico = [w[0] for w in modelo_topic_reducido.get_topic(topic)]
    if any(pal in palabras_topico for pal in palabras_irrelevantes):
        temas_a_eliminar.append(topic)

# Filtrar
df_temas_limpios = df_temas[~df_temas.Topic.isin(temas_a_eliminar)]

# Ordenar por frecuencia
df_temas_ordenados = df_temas_limpios.sort_values(by="Count", ascending=False)

# Exportar a Excel
df_temas_ordenados.to_excel("temas_bot_limpios.xlsx", index=False)

# Vista previa
print(df_temas_ordenados.head(10))
for topic_id in df_temas["Topic"]:
    print(f"Tema {topic_id}:")
    print(modelo_topic_reducido.get_topic(topic_id))
    print("\n")

etiquetas_manual = {
    -1: "Problemas generales con la cuenta",
     0: "Promociones de giros gratuitos",
     1: "Cambio de correo electrónico",
     2: "Peticiones de ayuda genéricas",
     3: "Reclamación por giros no entregados",
     4: "Problemas con bono de registro",
     5: "Dudas sobre apuestas deportivas o casino",
     6: "Preguntas sobre juegos y beneficios",
     7: "Problemas para retirar dinero",
     8: "Verificación de cuenta",
     9: "Dónde están mis giros / promociones",
    10: "Solicitud para hablar con un agente",
    11: "Consultas sobre promociones del día",
    12: "Bono de bienvenida / cómo usarlo",
    13: "Promoción de pronóstico sin riesgo",
    14: "Primer depósito / recarga inicial",
    15: "Reclamaciones por espera de 24-72h",
    16: "Promoción de cumpleaños",
    17: "Promoción Día del Padre",
    18: "Frases poco informativas"
}

df_temas = modelo_topic.get_topic_info()
df_temas["Etiqueta Manual"] = df_temas["Topic"].map(etiquetas_manual)
df_temas_ordenado = df_temas.sort_values(by="Count", ascending=False)
df_temas_ordenado.to_excel("temas_etiquetados.xlsx", index=False)

pip install transformers torch pysentimiento

from pysentimiento import create_analyzer
import pandas as pd

# Crear el analizador de sentimientos en español
analyzer = create_analyzer(task="sentiment", lang="es")

# Lista de textos (usa las mismas frases que usaste para BERTopic)
frases = frases_limpias  # lista de frases originales sin signos ni lematizar

# Analizar los sentimientos
resultados = [analyzer.predict(frase) for frase in frases]

# Extraer sentimiento y score
sentimientos = [r.output for r in resultados]
probabilidades = [r.probas for r in resultados]

# Crear DataFrame con los resultados
df_sentimientos = pd.DataFrame({
    "Frase": frases,
    "Sentimiento": sentimientos,
    "Probabilidad": [proba[max(proba, key=proba.get)] for proba in probabilidades]
})

# Ver resumen general
resumen = df_sentimientos["Sentimiento"].value_counts(normalize=True) * 100
print("Distribución de sentimientos (%):\n")
print(resumen.round(2))

# Exportar si lo deseas
df_sentimientos.to_excel("analisis_sentimientos.xlsx", index=False)

import pandas as pd

# Cargar archivos
df_temas = pd.read_excel("temas_bot_limpios.xlsx")
df_sentimientos = pd.read_excel("analisis_sentimientos.xlsx")

# Verificar contenido
print(df_temas.columns)
print(df_sentimientos.columns)

df_temas_completo = modelo_topic_reducido.get_document_info(frases_limpias)
df_temas_completo[["Document", "Topic"]].rename(columns={"Document": "Frase", "Topic": "TemaReducido"}).to_excel("temas_bot_limpios_completo.xlsx", index=False)

df_temas = pd.read_excel("temas_bot_limpios_completo.xlsx")
df_sentimientos = pd.read_excel("analisis_sentimientos.xlsx")

# Unir por la frase
df_completo = pd.merge(df_temas, df_sentimientos, on="Frase", how="inner")

# Agrupar por TemaReducido y Sentimiento
resumen = df_completo.groupby(["TemaReducido", "Sentimiento"]).size().unstack(fill_value=0)

# Calcular porcentaje por tema
resumen_pct = resumen.div(resumen.sum(axis=1), axis=0) * 100

# Ordenar por porcentaje de negativos
resumen_ordenado = resumen_pct.sort_values(by="NEG", ascending=False)

print(resumen_ordenado.round(2))

import matplotlib.pyplot as plt

# Graficar los primeros 10 temas con más negatividad
resumen_ordenado.head(10).plot(
    kind='barh',
    stacked=True,
    color=["red", "gray", "green"],
    figsize=(10, 6)
)
plt.xlabel("Porcentaje")
plt.ylabel("Tema Reducido")
plt.title("Distribución de Sentimientos por Tema Reducido")
plt.legend(title="Sentimiento", loc="lower right")
plt.tight_layout()
plt.show()
